# Examples

These examples are available in `notebooks/examples.ipynb` and more examples are available in the `tests/` directory. In both cases it is easiest to obtain these by downloading the source code.

## Initial setup

In [37]:
from atmodeller import (
    Species,
    InteriorAtmosphere,
    Planet,
    earth_oceans_to_hydrogen_mass,
    debug_logger,
)
from atmodeller.solubility import get_solubility_models
from atmodeller.thermodata import get_thermodata
from atmodeller.eos import get_eos_models
from atmodeller.thermodata import IronWustiteBuffer
from atmodeller.containers import ConstantFugacityConstraint
import logging
import numpy as np
import jax.numpy as jnp

logger = debug_logger()
logger.setLevel(logging.INFO)

# For more output use DEBUG
# logger.setLevel(logging.DEBUG)

## Species and thermodynamic data

The species available in *Atmodeller* can be found in the `thermodata` subpackage, where the suffix of the dictionary key describes the *states of aggregation* in accordance with the JANAF convention.

In [38]:
# Get all available species
thermodata = get_thermodata()
logger.info("Available species = %s", thermodata.keys())

# For example, get CO2 gas
CO2_g = thermodata["CO2_g"]
# Compute the Gibbs energy relative to RT at 2000 K
temperature = 2000.0
gibbs = CO2_g.get_gibbs_over_RT(temperature)
logger.info("Gibbs/RT = %s", gibbs)
# Compute the composition
composition = CO2_g.composition
logger.info("Composition = %s", composition)
# Etc., other methods are available to compute other quantities

[11:28:45 - atmodeller                     - INFO     ] - Available species = dict_keys(['C_g', 'CH4_g', 'Cl2_g', 'CO_g', 'CO2_g', 'C_cr', 'C2H2_g', 'COS_g', 'FeO_g', 'Fe_g', 'H2_g', 'H2O_cr', 'H2O_g', 'H2O_l', 'H2S_g', 'HCl_g', 'HCN_g', 'He_g', 'Mg_g', 'MgO_g', 'N2_g', 'NH3_g', 'NO_g', 'O2_g', 'OH_g', 'S_alpha', 'S_beta', 'S_l', 'S2_g', 'SH_g', 'Si_cr', 'Si_l', 'Si_g', 'SiO2_g', 'SiO_g', 'SiH4_g', 'SiO2_l', 'SO_g', 'SO2_g', 'SO3_g', 'Ar_g'])
[11:28:45 - atmodeller                     - INFO     ] - Gibbs/RT = -55.36417507969985
[11:28:45 - atmodeller                     - INFO     ] - Composition = ImmutableMap({'C': (1, 12.01074, 0.27291212929920894), 'O': (2, 31.99881, 0.7270878707007911)})


## Solubility

Solubility laws are available in the `solubility` subpackage.

In [39]:
solubility_models = get_solubility_models()
logger.info("Solubility models = %s", solubility_models.keys())

CO2_basalt = solubility_models["CO2_basalt_dixon95"]
# Compute the concentration at fCO2=0.5 bar, 1300 K, and 1 bar
# Note that fugacity is the first argument and others are keyword only
concentration = CO2_basalt.concentration(0.5, temperature=1300, pressure=1)
logger.info("Concentration (ppmw) = %s", concentration)

[11:28:50 - atmodeller                     - INFO     ] - Solubility models = dict_keys(['CH4_basalt_ardia13', 'CO2_basalt_dixon95', 'CO_basalt_armstrong15', 'CO_basalt_yoshioka19', 'CO_rhyolite_yoshioka19', 'Cl2_ano_dio_for_thomas21', 'Cl2_basalt_thomas21', 'H2O_ano_dio_newcombe17', 'H2O_basalt_dixon95', 'H2O_basalt_mitchell17', 'H2O_lunar_glass_newcombe17', 'H2O_peridotite_sossi23', 'H2_andesite_hirschmann12', 'H2_basalt_hirschmann12', 'H2_silicic_melts_gaillard03', 'He_basalt_jambon86', 'N2_basalt_bernadou21', 'N2_basalt_dasgupta22', 'N2_basalt_libourel03', 'S2_andesite_boulliung23', 'S2_basalt_boulliung23', 'S2_sulfate_andesite_boulliung23', 'S2_sulfate_basalt_boulliung23', 'S2_sulfate_trachybasalt_boulliung23', 'S2_sulfide_andesite_boulliung23', 'S2_sulfide_basalt_boulliung23', 'S2_sulfide_trachybasalt_boulliung23', 'S2_trachybasalt_boulliung23', 'NO_SOLUBILITY'])
[11:28:50 - atmodeller                     - INFO     ] - Concentration (ppmw) = 0.22841535272000957


In [64]:
N2_basalt = solubility_models["N2_basalt_libourel03"]
# Compute the concentration at fCO2=0.5 bar, 1300 K, and 1 bar
# Note that fugacity is the first argument and others are keyword only
concentration = N2_basalt.concentration(0.20, temperature=1698.15, pressure=1, fO2=10**-16.2)
logger.info("Concentration (ppmw) = %s", concentration)

[15:00:43 - atmodeller                     - INFO     ] - Concentration (ppmw) = 377.1406984833235


In [62]:
N2_basalt_dasgupta = solubility_models["N2_basalt_dasgupta22"]
# Compute the concentration at fCO2=0.5 bar, 1300 K, and 1 bar
# Note that fugacity is the first argument and others are keyword only
concentration = N2_basalt_dasgupta.concentration(
    1550, temperature=1773.15, pressure=1708.7, fO2=1.8e-13
)
logger.info("Concentration (ppmw) = %s", concentration)

[12:06:06 - atmodeller                     - INFO     ] - Concentration (ppmw) = 1006.8742354192402


In [45]:
def IWBuffer_H21(T, P_bar):
    # T in Kelvin, P_bar in bar
    P_GPa = P_bar / 1e4
    a = 6.844864 + (1.175691e-1 * P_GPa) + (1.143873e-3 * (P_GPa**2))
    b = 5.791364e-4 - (2.891434e-4 * P_GPa) - (2.737171e-7 * (P_GPa**2))
    c = (
        -7.971469e-5
        + (3.198005e-5 * P_GPa)
        + (1.059554e-10 * (P_GPa**3))
        + (2.014461e-7 * (P_GPa**0.5))
    )
    d = -2.769002e4 + (5.285977e2 * P_GPa) - (2.919275 * (P_GPa**2))
    logfO2 = a + (b * T) + (c * T * np.log(T)) + d / T
    return logfO2

In [59]:
IWBuffer_test = IWBuffer_H21(1773.15, 1708.7)
print(10 ** (IWBuffer_test - 4))

1.799329010613709e-13


## Real gas EOS

Real gas equations of state are available in the `eos` subpackage.

In [4]:
# Get all available EOS models
eos_models = get_eos_models()
logger.info("EOS models = %s", eos_models.keys())

# Get a CH4 model
CH4_eos_model = eos_models["CH4_beattie_holley58"]
# Compute the fugacity at 800 K and 100 bar
fugacity = CH4_eos_model.fugacity(800, 100)
logger.info("Fugacity = %s bar", fugacity)
# Compute the compressibility factor at the same conditions
compressibility = CH4_eos_model.compressibility_factor(800, 100)
logger.info("Compressibility factor = %s", compressibility)
# Etc., other methods are available to compute other quantities

[11:24:46 - atmodeller                     - INFO     ] - EOS models = dict_keys(['H2_chabrier21', 'H2_He_Y0275_chabrier21', 'H2_He_Y0292_chabrier21', 'H2_He_Y0297_chabrier21', 'He_chabrier21', 'CH4_beattie_holley58', 'CO2_beattie_holley58', 'H2_beattie_holley58', 'He_beattie_holley58', 'N2_beattie_holley58', 'NH3_beattie_holley58', 'O2_beattie_holley58', 'CH4_cork_cs_holland91', 'CO_cork_cs_holland91', 'CO2_cork_holland91', 'CO2_cork_holland98', 'CO2_cork_cs_holland91', 'H2_cork_cs_holland91', 'H2O_cork_holland91', 'H2O_cork_holland98', 'H2S_cork_cs_holland11', 'N2_cork_cs_holland91', 'S2_cork_cs_holland11', 'Ar_cs_saxena87', 'CH4_cs_shi92', 'CO_cs_shi92', 'CO2_cs_shi92', 'COS_cs_shi92', 'H2_shi92', 'H2S_shi92', 'N2_cs_saxena87', 'O2_cs_shi92', 'S2_cs_shi92', 'SO2_shi92'])
[11:24:47 - atmodeller                     - INFO     ] - Fugacity = 103.31104964171287 bar
[11:24:47 - atmodeller                     - INFO     ] - Compressibility factor = 1.0336552056370774


## Model with mass constraints

A common scenario is to calculate how volatiles partition between a magma ocean and an atmosphere when the total elemental abundances are constrained. `Planet()` defaults to a molten Earth, but the planetary parameters can be changed using input arguments.

In [34]:
solubility_models = get_solubility_models()

H2_g = Species.create_gas("H2_g")
H2O_g = Species.create_gas("H2O_g", solubility=solubility_models["H2O_peridotite_sossi23"])
O2_g = Species.create_gas("O2_g")

species = (H2_g, H2O_g, O2_g)

# Planet has input arguments that you can change. See the class documentation.
planet = Planet()
interior_atmosphere = InteriorAtmosphere(species)

oceans = 1
h_kg = earth_oceans_to_hydrogen_mass(oceans)
o_kg = 6.25774e20
mass_constraints = {
    "H": h_kg,
    "O": o_kg,
}
fugacity_constraints = {
    H2_g.name: ConstantFugacityConstraint(jnp.array([1.0])),
    H2O_g.name: ConstantFugacityConstraint(jnp.array([2.0])),
}
# If you do not specify an initial solution guess then a default will be used
# Initial solution guess number density (molecules/m^3)
# initial_log_number_density = 50 * np.ones(len(species), dtype=np.float_)

interior_atmosphere.initialise_solve(
    planet=planet,
    # initial_log_number_density=initial_log_number_density,
    # mass_constraints=mass_constraints,
    fugacity_constraints=fugacity_constraints,
)
output = interior_atmosphere.solve()

# Quick look at the solution
solution = output.quick_look()
logger.info("solution = %s", solution)

# Get complete solution as a dictionary
# solution_asdict = output.asdict()
# logger.info(solution_asdict)

# Get the complete solution as dataframes
# solution_dataframes = output.to_dataframes()

# Write the complete solution to Excel
# output.to_excel("example_single")

[13:53:22 - atmodeller.classes             - INFO     ] - species = ['H2_g', 'H2O_g', 'O2_g']
[13:53:22 - atmodeller.classes             - INFO     ] - reactions = {0: '2.0 H2O_g = 2.0 H2_g + 1.0 O2_g'}
[13:53:23 - atmodeller.classes             - INFO     ] - Compile time: 0.554042 seconds
[13:53:23 - atmodeller.classes             - INFO     ] - Execution time: 0.000542 seconds
[13:53:23 - atmodeller.output              - INFO     ] - Creating Output
[13:53:23 - atmodeller                     - INFO     ] - solution = {'H2_g': 1.0, 'H2_g_activity': 1.0, 'H2O_g': 2.0000000000000036, 'H2O_g_activity': 2.0000000000000036, 'O2_g': 3.3206881972108655e-07, 'O2_g_activity': 3.3206881972108655e-07}


In [36]:
Ar_g = Species.create_gas("Ar_g")
SO2_g = Species.create_gas("SO2_g")
O2_g = Species.create_gas("O2_g")
S2_g = Species.create_gas("S2_g")
SO_g = Species.create_gas("SO_g")

species = (Ar_g, SO2_g, O2_g, S2_g, SO_g)

# Planet has input arguments that you can change. See the class documentation.
planet = Planet()
interior_atmosphere = InteriorAtmosphere(species)

oceans = 1
s_kg = earth_oceans_to_hydrogen_mass(oceans)
o_kg = 6.25774e20
ar_kg = 6.25774e20

so2_f = 0.04
ar_f = 0.959
o2_f = 1e-18
fugacity_constraints = {
    SO2_g.name: ConstantFugacityConstraint(jnp.array([0.04])),
    Ar_g.name: ConstantFugacityConstraint(jnp.array([0.96])),
}

mass_constraints = {"Ar": ar_kg, "O": o_kg, "S": s_kg}

# If you do not specify an initial solution guess then a default will be used
# Initial solution guess number density (molecules/m^3)
# initial_log_number_density = 50 * np.ones(len(species), dtype=np.float_)

interior_atmosphere.initialise_solve(
    planet=planet,
    # initial_log_number_density=initial_log_number_density,
    fugacity_constraints=fugacity_constraints,
    # mass_constraints = mass_constraints,
)
output = interior_atmosphere.solve()

# Quick look at the solution
solution = output.quick_look()
logger.info("solution = %s", solution)

# Get complete solution as a dictionary
# solution_asdict = output.asdict()
# logger.info(solution_asdict)

# Get the complete solution as dataframes
# solution_dataframes = output.to_dataframes()

# Write the complete solution to Excel
# output.to_excel("example_single")

[14:06:57 - atmodeller.classes             - INFO     ] - species = ['Ar_g', 'O2S_g', 'O2_g', 'S2_g', 'OS_g']
[14:06:57 - atmodeller.classes             - INFO     ] - reactions = {0: '2.0 O2S_g = 2.0 O2_g + 1.0 S2_g', 1: '1.0 O2S_g = 0.5 O2_g + 1.0 OS_g'}
[14:06:58 - atmodeller.classes             - INFO     ] - Compile time: 0.865703 seconds
[14:06:58 - atmodeller.classes             - INFO     ] - Execution time: 0.000560 seconds
[14:06:58 - atmodeller.output              - INFO     ] - Creating Output
[14:06:58 - atmodeller                     - INFO     ] - solution = {'Ar_g': 0.9599999999999977, 'Ar_g_activity': 0.9599999999999977, 'O2S_g': 0.04000000000000006, 'O2S_g_activity': 0.04000000000000006, 'O2_g': 9.149832047767465e-06, 'O2_g_activity': 9.149832047767465e-06, 'S2_g': 0.00010497907047397462, 'S2_g_activity': 0.00010497907047397462, 'OS_g': 0.002022681363670241, 'OS_g_activity': 0.002022681363670241}


## Batch calculation

For a batch calculation you can provide arrays to the planet or constraints. All arrays must have the same size because for a batch calculation the array values are aligned by position. Single values will automatically be broadcasted to the maximum array size.

In [ ]:
solubility_models = get_solubility_models()

H2_g = Species.create_gas("H2_g")
H2O_g = Species.create_gas("H2O_g", solubility=solubility_models["H2O_peridotite_sossi23"])
O2_g = Species.create_gas("O2_g")

species = (H2_g, H2O_g, O2_g)

# Batch temperature and radius, where the entries correspond by position. You could also choose
# to leave one or both as scalars.
# You must specify dtype=np.float_ for surface temperature
surface_temperature = np.array([2000, 2000, 1500, 1500], dtype=np.float_)
surface_radius = 6371000.0 * np.array([1.5, 3, 1.5, 3], dtype=np.float_)

planet = Planet(surface_temperature=surface_temperature, surface_radius=surface_radius)
interior_atmosphere = InteriorAtmosphere(species)

oceans = 1
h_kg = earth_oceans_to_hydrogen_mass(oceans)
o_kg = 6.25774e20
scale_factor = 5
mass_constraints = {
    # We can also batch constraints, as long as we also have a total of 4 entries
    "H": np.array([h_kg, h_kg, h_kg * scale_factor, h_kg * scale_factor], dtype=np.float_),
    "O": np.array([o_kg, o_kg * scale_factor, o_kg, o_kg * scale_factor], dtype=np.float_),
}

# Initial solution guess number density (molecules/m^3)
initial_log_number_density = 50 * np.ones(len(species), dtype=np.float_)

interior_atmosphere.initialise_solve(
    planet=planet,
    initial_log_number_density=initial_log_number_density,
    mass_constraints=mass_constraints,
)
output = interior_atmosphere.solve()

# Quick look at the solution
# solution = output.quick_look()
# logger.info("Quick look = %s", solution)

# Get complete solution as a dictionary
# solution_asdict = output.asdict()
# logger.info(solution_asdict)

# Write the complete solution to Excel
# output.to_excel("example_batch")

## Time integration

For models where you need to dynamically update constraints during the course of a time-integration, atmodeller can be utilised as follows. The model is pre-compiled using `initialise_solve` and then the compiled function is subsequently called with different arguments. Note that the order of the arguments and the size of the arrays must be the same as those used to initialise the model, but of course the values can be different.

In [ ]:
# This first part is the initialisation stage and should appear outside of your main time loop

solubility_models = get_solubility_models()

H2_g = Species.create_gas("H2_g")
H2O_g = Species.create_gas("H2O_g", solubility=solubility_models["H2O_peridotite_sossi23"])
O2_g = Species.create_gas("O2_g")

species = (H2_g, H2O_g, O2_g)
planet = Planet()
interior_atmosphere = InteriorAtmosphere(species)

oceans = 1
h_kg = earth_oceans_to_hydrogen_mass(oceans)
o_kg = 6.25774e20
mass_constraints = {
    "H": h_kg,
    "O": o_kg,
}

# Initial solution guess number density (molecules/m^3)
initial_log_number_density = 50 * np.ones(len(species), dtype=np.float_)

# Precompile
interior_atmosphere.initialise_solve(
    planet=planet,
    initial_log_number_density=initial_log_number_density,
    mass_constraints=mass_constraints,
)

# This is the time loop, where something changes and you want to re-solve using Atmodeller
for ii in range(1, 4):
    # Let's say we update the mass constraints. The number of constraints and the value size must
    # remain the same as the initialised model, but you are free to update their values. Here,
    # scale by number of earth oceans for the hydrogen mass.
    logger.info("Iteration %d", ii)
    logger.info("Your code does something here to compute new masses")
    mass_constraints = {"H": h_kg * ii, "O": o_kg}
    # These solves are fast because they use the JAX-compiled code
    logger.info("Atmodeller solve using JIT compiled code")
    output = interior_atmosphere.solve(mass_constraints=mass_constraints)

    # Quick look at the solution
    solution = output.quick_look()
    logger.info("solution = %s", solution)

    # Get complete solution as a dictionary
    # If required, get complete output to feedback into other calculations during the time loop
    # solution_asdict = output.asdict()

## Monte Carlo

Exploring atmospheric compositions in a Monte Carlo model can be achieved with a batch 
calculation over a range of parameters. Note that in this case the same initial solution is used 
for all cases. For a large parameter range you may need to run several batches with different 
initial solutions to prevent the solver from failing, and then stitch the output together.

In [ ]:
solubility_models = get_solubility_models()

H2_g = Species.create_gas("H2_g")
H2O_g = Species.create_gas("H2O_g", solubility=solubility_models["H2O_peridotite_sossi23"])
O2_g = Species.create_gas("O2_g")

species = (H2_g, H2O_g, O2_g)
planet = Planet()
interior_atmosphere = InteriorAtmosphere(species)

number_of_realisations = 1000
log10_number_oceans = np.random.uniform(0, 3, number_of_realisations)
number_oceans = 10**log10_number_oceans
fO2_min = -3
fO2_max = 3
fO2_log10_shifts = np.random.uniform(fO2_min, fO2_max, number_of_realisations)

oceans = 1
h_kg = earth_oceans_to_hydrogen_mass(number_oceans)
mass_constraints = {
    "H": h_kg,
}
fugacity_constraints = {O2_g.name: IronWustiteBuffer(fO2_log10_shifts)}

# Initial solution guess number density (molecules/m^3)
initial_log_number_density = 50 * np.ones(len(species), dtype=np.float_)

# Precompile
interior_atmosphere.initialise_solve(
    planet=planet,
    initial_log_number_density=initial_log_number_density,
    mass_constraints=mass_constraints,
    fugacity_constraints=fugacity_constraints,
)
output = interior_atmosphere.solve()

# Quick look at the solution
# solution = output.quick_look()

# Get complete solution as a dictionary
# solution_asdict = output.asdict()
# logger.info(solution_asdict)

# Write the complete solution to Excel
# output.to_excel("example_monte_carlo")